Can follow some code in timm's dem merging function.
https://github.com/accs-uaa/beringian-vegetation/blob/master/package_GeospatialProcessing/mergeElevationTiles.py

Recommended workflow for geoprocessing:
- import arcpy
- set environments
- create lists of gis objects
- iterate through list elements
- execute geoprocessing tools

Geoprocessing steps in this script
1. create a set of folders and gdb for each region under a master AKSSF folder
2. loop through regions and grids or feature classes (e.g. catchments or flowlines)
3. merge grids or feature classes
4. save in folder/gdb with consistent name (e.g. fac_merge)

These seamed grids will enable us to use zonal statistics with the selected catchments
and watersheds to get our environmental variables for modeling.

# step 1
# Create folders for storing outputs

#import modules
import os
import arcpy

#user inputs
baseFolder = "W:\\GIS\\"
projectName = "AKSSF"

#define folder paths
ciFolder = baseFolder + "\\" + projectName + "\\"+ "Cook_Inlet"
pwsFolder = baseFolder + "\\" + projectName + "\\"+ "Prince_William_Sound"
bbFolder = baseFolder + "\\" + projectName + "\\"+ "Bristol_Bay"
crFolder = baseFolder + "\\" + projectName + "\\"+ "Copper_River"
kodFolder = baseFolder + "\\" + projectName + "\\"+ "Kodiak"

#create list of folder paths
folders = [ciFolder, pwsFolder, bbFolder, crFolder, kodFolder]

#check if project folder exists
if not os.path.exists(baseFolder + projectName):
    #Create folder if it does not exist
    for folder in folders:
        os.makedirs(folder)
        print("Created: " + folder)
else:
    #if project folder exists, print error message
    print (projectName + " exists. Please provide a new project name.")

In [1]:
# step 2
# merge catchments and vaa tables
import arcpy
import os

baseFolder = "W:\\GIS\\"
ciNHD = baseFolder + "NHDPlus\\CookInlet_20201216"
arcpy.env.workspace = ciNHD
gdbs = arcpy.ListWorkspaces('NHDPLUS*','FileGDB')
print(gdbs)

ciCats = []
ciVAA = []
for gdb in gdbs:
    arcpy.env.workspace = gdb
    catpath = os.path.join(arcpy.env.workspace, "NHDPlus\\NHDPlusCatchment")
    ciCats.append(catpath)
    vaapath = os.path.join(arcpy.env.workspace, "NHDPlusFlowlineVAA")
    ciVAA.append(vaapath)

arcpy.env.workspace =  r"W:/GIS/AKSSF/Cook_Inlet.gdb"
arcpy.env.outputCoordinateSystem = arcpy.Describe(ciCats[1]).spatialReference
arcpy.env.overwriteOutput = True

arcpy.management.Merge(ciCats, r"W:/GIS/AKSSF/Cook_Inlet/Cook_Inlet.gdb/cats_merge")
arcpy.management.Merge(ciVAA, r"W:/GIS/AKSSF/Cook_Inlet/Cook_Inlet.gdb/vaa_merge")



['W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020202_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020301_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020302_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020401_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020402_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020501_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020502_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020503_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020504_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020505_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020601_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020602_HU8_GDB.gdb', 'W:\\GIS\\NHDPlus\\CookInlet_20201216\\NHDPLUS_H_19020800_HU8_GDB.gdb']


<Result 'W:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\vaa_merge'>

In [ ]:
# step 3
# merge raster datasets, dems, flow accumulation grids, others?
# elevation grid - elevation statistics
# flow accumulation grid - watershed size
# slope grid (create first)
# aspect grid (create first - Dustin has a script with this)


import arcpy
import os

arcpy.env.workspace = r"T:\Aquatic\AKSSF\NHDPlus\CookInlet_20201216"
flowacc_rasters = []
raster_folders = arcpy.ListWorkspaces("*Rasters*")
print(raster_folders)

for folder in raster_folders:
    arcpy.env.workspace = folder
    rasters = arcpy.ListRasters()
    for raster in rasters:
        if raster == 'fac.tif':
            flowacc_raspath = os.path.normpath(os.path.join(folder, raster))
            flowacc_rasters.append(flowacc_raspath)

print(flowacc_rasters)
print(flowacc_rasters[1])
dataset = flowacc_rasters[1]
spatial_ref = arcpy.Describe(dataset).spatialReference

arcpy.MosaicToNewRaster_management(flowacc_rasters, r"W:\GIS\AKSSF Southcentral\Temporary", "ci_fac_merge.tif", spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")